In [ ]:
from github import Github
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt

url = "HikaGenji/hl-dl"
g = Github()  # No authentication needed for public repos, but you can add a token for higher rate limits
repo = g.get_repo(url)
contents = repo.get_contents("data")
files = [content_file.path for content_file in contents]

In [ ]:
import duckdb

parquet_url = [f"https://github.com/{url}/raw/main/{f}" for f in files]

# Example query: select columns and filter rows
result = duckdb.query(f"SELECT * FROM read_parquet({parquet_url})").pl()

print(result.head())


shape: (5, 17)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ timestamp ┆ ethAddres ┆ accountVa ┆ displayNa ┆ … ┆ month_vol ┆ allTime_p ┆ allTime_r ┆ allTime_ │
│ ---       ┆ s         ┆ lue       ┆ me        ┆   ┆ ume       ┆ nl        ┆ oi        ┆ volume   │
│ str       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---      │
│           ┆ str       ┆ f64       ┆ str       ┆   ┆ f64       ┆ f64       ┆ f64       ┆ f64      │
╞═══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪══════════╡
│ 2025-06-0 ┆ 0x162cc7c ┆ 2.4187e7  ┆ ABC       ┆ … ┆ 4.1088e10 ┆ 2.0885e7  ┆ 1.394246  ┆ 1.2319e1 │
│ 8T08:44:1 ┆ 861ebd0c0 ┆           ┆           ┆   ┆           ┆           ┆           ┆ 1        │
│ 6.170503  ┆ 6b3d72319 ┆           ┆           ┆   ┆           ┆           ┆           ┆          │
│           ┆ 201…      ┆           ┆           ┆   ┆           ┆           

In [42]:
df = result.select('timestamp', 'ethAddress', pl.col('allTime_pnl').diff().over(partition_by='ethAddress').fill_null(0).alias('hour_pnl'),
                             pl.col('allTime_volume').diff().over(partition_by='ethAddress').fill_null(0).alias('hour_volume'))
                             

In [43]:
stats = df.group_by('ethAddress').agg(pl.col('hour_pnl').mean().alias('mean_hour_pnl'), 
                              pl.col('hour_pnl').std().alias('std_hour_pnl'),
                              pl.col('hour_volume').sum().alias('volume')).\
                              with_columns(pl.col('mean_hour_pnl').truediv(pl.col('std_hour_pnl')).alias('sharpe')).\
                              filter(pl.col('sharpe').is_not_null(),
                                     pl.col('sharpe').is_not_nan()).\
                              sort('sharpe', descending=True)

In [49]:
stats.filter(pl.col('volume')>0, pl.col('sharpe')>0.5)

ethAddress,mean_hour_pnl,std_hour_pnl,volume,sharpe
str,f64,f64,f64,f64
"""0x4fbc1cd366c5a9203a878089f9b0…",6330.621099,6518.952045,5.4406e6,0.97111
"""0x44cd38994fbf89db3f7fa9465ca4…",397.573815,456.827797,605448.99097,0.870293
"""0xdcac85ecae7148886029c20e661d…",18531.973273,23185.448434,4.7529e6,0.799293
"""0x7c4b71c1d8a11ed426f301c372a7…",2886.210969,3723.374084,1.3667e7,0.77516
"""0x67ce2b979ee9615cdf634e6043ac…",92.004636,122.047604,824.148,0.753842
…,…,…,…,…
"""0xc070a046a5e4e1d16f6fe340cca0…",32.492991,64.016802,125437.0,0.50757
"""0x200159c7f753fbd7c08f16382746…",0.803554,1.584439,393.035488,0.507154
"""0xc66f4198c3b132b38017d76c6fed…",0.823468,1.624503,1518.13571,0.506905
